**remark**
trying to create a file which can be used to calculate the CIPW norm beginning from an excell file (used the infile previously

In [1]:
import numpy as np
import pandas as pd
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc

C:\Users\u0108248\AppData\Local\Continuum\miniconda3\envs\main\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [ ]:
CIPW_try = pd.read_excel("../_CIPW/ESM_try.xlsx", index=None)

In [ ]:
CIPW_try = preproc.replace_nan(CIPW_try, "")

In [ ]:
CIPW_try

In [5]:
CIPW_try1 = CIPW_try["Unnamed: 0"].map(str) + " " + CIPW_try["areax"].map(str) + " " +  "0" + " " + "1" + " " + CIPW_try["Unnamed: 2"].map(str) + "  " + CIPW_try["Unnamed: 3"].map(str) + "  " + CIPW_try["Unnamed: 4"].map(str) + "  " + CIPW_try["Unnamed: 5"].map(str) + "  " + CIPW_try["Unnamed: 6"].map(str) + "  " + CIPW_try["Unnamed: 7"].map(str) + "  " + CIPW_try["Unnamed: 8"].map(str) + "  " + CIPW_try["Unnamed: 9"].map(str) + "  " + CIPW_try["Unnamed: 10"].map(str) + "  " + CIPW_try["Unnamed: 11"].map(str) + "  " + CIPW_try["Unnamed: 12"].map(str) + "  " + CIPW_try["Unnamed: 13"].map(str) + "  " + CIPW_try["Unnamed: 14"].map(str) + "  " + CIPW_try["Unnamed: 15"].map(str) + "  " + CIPW_try["Unnamed: 16"].map(str) + " " + "0.0" + " " + "0.0" + " " + "0.0" + " " + "0.0" + " " + "0.0"

In [7]:
CIPW_try1[0]

'SIO2 TIO2 AL2O3 FE2O3 FEO MNO MGO CAO NA2O K2O P2O5 CO2 F S BAO                                                                                   0 1                              0.0 0.0 0.0 0.0 0.0'

In [ ]:
CIPW_try.to_csv()

In [44]:
CIPW_try1.to_excel("../_CIPW/ESM.xlsx")

## Test - convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one**

In [20]:
# Only iterested in sample number and oxides
mineralogy = pd.read_excel("../_PETRO/classification_altereddata.xlsx", index_col=0, usecols=['Unnamed: 0', 'SiO2', 'TiO2', 'Al2O3', 'Fe2O3',
       'FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'l.i.', 'oth'])
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'P2O5', 'l.i.', 'oth'],
      dtype='object')

In [21]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [22]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["l.i.", "oth"], axis=1))

In [23]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [24]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
1,80.888978,0.040044,10.171188,0.610672,1.721894,0.010011,0.400440,0.550606,2.002202,3.593953,0.010011
2,80.685830,0.100857,10.186586,0.171457,0.564801,0.020171,0.403429,0.353001,2.319718,5.143722,0.050429
3,80.776228,0.050536,9.995957,0.161714,1.748534,0.020214,0.121286,0.141500,0.758035,6.215888,0.010107
4,79.316869,0.039833,9.599681,1.145190,0.746863,0.099582,0.448118,0.667198,3.694483,4.232225,0.009958
5,80.231026,0.081062,10.375925,0.648495,2.634512,0.040531,0.050664,1.266592,1.540176,3.120884,0.010133


In [14]:
def convert_to_CIPWFULL_format(df, path, dataset_name="Dataset", index_prefix=None, rock_type="P", 
                               rock_suite_column=None, normalization=False):
    """Converts a pandas DataFrame to a txt file compatible 
    as an input file for the CIPWFULL program by AL-Mishwat (2015)
    
    Parameters:
    -----------
    df : pd.DataFrame
        mineralogy data in the form of oxides
    path : str
        path to where to save resulting file
    dataset_name : str (optional)
        Name opf dataset to use in CIPWFULL
        defaults to 'Dataset'
    index_prefix : str (optional)
        Prefix to use before index
        defaults to None
    rock_type : str (optional)
        Rock type indicator:
            'P' for plutonic
            'V' for volcanic
        defaults to 'P'
    rock_suite_column : str (optional)
        Which column name to use a 'rock suite' in CIPWFULL
        The rock suite acts as a grouping variable
        defaults to None
    normalization : bool (optional)
        Write flag to file whether to let CIPWFULL 
        normalize the data before norm calculation
        defaults to False
        
    Returns:
    --------
    df_CIPW : pd.DataFrame
        altered mineralogy data according to CIPWFULL input format
    
    (file is written to disk)
    
    """
    
    # Get copy of original df so that it doesn't get altered
    df_CIPW = df.copy()
    
    # Add prefix to index if required
    if index_prefix:
        df_CIPW.index = f"{index_prefix}" + df_CIPW.index.astype(str)
        
    # Add additional columns to be moved to index
    df_CIPW["Type"] = "P"
    df_CIPW["Cancrinite"] = 0
    if rock_suite_column:
        df_CIPW["Rock suite"] = df_CIPW[rock_suite_column]
        df_CIPW.drop(rock_suite_column, axis=1)
    else:
        df_CIPW["Rock suite"] = 1 # This could be changed to a grouping variable such as geographic area, time
        
    # Move additional columns to index
    df_CIPW = df_CIPW.set_index(["Type", "Cancrinite", "Rock suite"], append=True)
    
    # Start writing file (file saving/closing is handled automatically)
    with open(path, 'w') as f:

        # Write a '1' before the dataset name if you want CIPWFULL 
        # to normalize the data before the calculation; else write space
        if normalization:
            f.write("1")
        else:
            f.write(" ")

        # Write dataset name
        f.write(f"{dataset_name}\n")

        # Write column names and index+data
        # The floating point number need to be formatted so as not 
        # to go over the 120 character line limit of CIPWFULL
        df_CIPW.to_csv(f, mode='w', sep=" ", line_terminator="\n", 
                       index_label=False, float_format="%2.5f")

        # Write '0' to indicate end of data entries
        f.write("0")

        # Write rock suite names if needed
        if rock_suite_column:
            f.write("\n")
            f.write(rock_suite_column.unique())
            f.write("\n")

In [16]:
convert_to_CIPWFULL_format(mineralogy, "../_CIPW/test.txt", dataset_name="Vistelius (1995)")

_____